In [16]:
from bs4 import BeautifulSoup
import numpy as np
import os
import re
import glob
import pandas as pd
import pathlib
from pandarallel import pandarallel
import tarfile
import unicodedata
from unidecode import unidecode

def unicodetoascii(text):
    TEXT = (text.
    		replace('\\xe2\\x80\\x99', "'").
            replace('\\xc3\\xa9', 'e').
            replace('\\xe2\\x80\\x90', '-').
            replace('\\xe2\\x80\\x91', '-').
            replace('\\xe2\\x80\\x92', '-').
            replace('\\xe2\\x80\\x93', '-').
            replace('\\xe2\\x80\\x94', '-').
            replace('\\xe2\\x80\\x94', '-').
            replace('\\xe2\\x80\\x98', "'").
            replace('\\xe2\\x80\\x9b', "'").
            replace('\\xe2\\x80\\x9c', '"').
            replace('\\xe2\\x80\\x9c', '"').
            replace('\\xe2\\x80\\x9d', '"').
            replace('\\xe2\\x80\\x9e', '"').
            replace('\\xe2\\x80\\x9f', '"').
            replace('\\xe2\\x80\\xa6', '...').#
            replace('\\xe2\\x80\\xb2', "'").
            replace('\\xe2\\x80\\xb3', "'").
            replace('\\xe2\\x80\\xb4', "'").
            replace('\\xe2\\x80\\xb5', "'").
            replace('\\xe2\\x80\\xb6', "'").
            replace('\\xe2\\x80\\xb7', "'").
            replace('\\xe2\\x81\\xba', "+").
            replace('\\xe2\\x81\\xbb', "-").
            replace('\\xe2\\x81\\xbc', "=").
            replace('\\xe2\\x81\\xbd', "(").
            replace('\\xe2\\x81\\xbe', ")"))
    return TEXT

def get_csv_size(csv_name):
    get_size = os.path.getsize(os.getcwd() + '/'+csv_name)
    mb_size = get_size/(1024 * 1024)
    mb_size = round(mb_size,1)
    return mb_size

In [1]:
df = pd.DataFrame(columns=['path','source'])
df.path = [str(x) for x in pathlib.Path('./volumes/').glob('**/*')]
df.source = df.path.apply(lambda x:x.split('/')[1].replace('.xml',''))
# df.source = df.filename.apply(lambda x:x.split('_')[0]) +'_eng'
# df['output'] = '3.Gigaword/upload/'+df.source +'/'+ df.filename
df

,path,source
0,volumes/frus1961-63v10-12mSupp.xml,frus1961-63v10-12mSupp
1,volumes/frus1964-68v29p2.xml,frus1964-68v29p2
2,volumes/frus1955-57v06.xml,frus1955-57v06
3,volumes/frus1865p1.xml,frus1865p1
4,volumes/frus1935v03.xml,frus1935v03
...,...,...
537,volumes/frus1928v02.xml,frus1928v02
538,volumes/frus1901.xml,frus1901
539,volumes/frus1964-68v21.xml,frus1964-68v21
540,volumes/frus1977-80v09Ed2.xml,frus1977-80v09Ed2


In [2]:
def extract_docs(filename):
    with open(filename, 'r') as f:
        data = f.read()

    soup = BeautifulSoup(data, "xml")
    docs = soup.find_all('div', type="document")
    docs= [str(i) for i in docs]
    return docs

pandarallel.initialize(nb_workers=38,progress_bar=True) 
df['docs'] = df.path.parallel_apply(extract_docs)

INFO: Pandarallel will run on 38 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [4]:
df = df.explode('docs').reset_index(drop=True)
df = df[~df.docs.isnull()].reset_index(drop=True)
df

,path,source,docs
0,volumes/frus1961-63v10-12mSupp.xml,frus1961-63v10-12mSupp,"<div frus:doc-dateTime-max=""1961-02-09T23:59:5..."
1,volumes/frus1961-63v10-12mSupp.xml,frus1961-63v10-12mSupp,"<div frus:doc-dateTime-max=""1961-03-03T23:59:5..."
2,volumes/frus1961-63v10-12mSupp.xml,frus1961-63v10-12mSupp,"<div frus:doc-dateTime-max=""1961-04-13T23:59:5..."
3,volumes/frus1961-63v10-12mSupp.xml,frus1961-63v10-12mSupp,"<div frus:doc-dateTime-max=""1961-07-18T23:59:5..."
4,volumes/frus1961-63v10-12mSupp.xml,frus1961-63v10-12mSupp,"<div frus:doc-dateTime-max=""1961-07-21T23:59:5..."
...,...,...,...
309567,volumes/frus1872p2v4.xml,frus1872p2v4,"<div frus:doc-dateTime-max=""1872-12-01T23:55:5..."
309568,volumes/frus1872p2v4.xml,frus1872p2v4,"<div frus:doc-dateTime-max=""1872-06-06T23:59:5..."
309569,volumes/frus1872p2v4.xml,frus1872p2v4,"<div frus:doc-dateTime-max=""1872-12-31T18:50:5..."
309570,volumes/frus1872p2v4.xml,frus1872p2v4,"<div frus:doc-dateTime-max=""1872-12-31T18:42:5..."


In [5]:
df.to_csv('frus_raw.csv', index=False)

# 2

In [8]:
def extract_doc(doc):
    soup = BeautifulSoup(doc, "xml")
    try:
        info = soup.div.attrs
        text = soup.text
        return pd.Series([info, text])
    except:
        return pd.Series([None]*2)


pandarallel.initialize(progress_bar=True)
df[['information','text']] = df['docs'].parallel_apply(extract_doc).values

INFO: Pandarallel will run on 36 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [9]:
df

,path,source,docs,information,text
0,volumes/frus1961-63v10-12mSupp.xml,frus1961-63v10-12mSupp,"<div frus:doc-dateTime-max=""1961-02-09T23:59:5...",{'doc-dateTime-max': '1961-02-09T23:59:59-05:0...,"\n1. Telegram 430 from La Paz, February 9Trans..."
1,volumes/frus1961-63v10-12mSupp.xml,frus1961-63v10-12mSupp,"<div frus:doc-dateTime-max=""1961-03-03T23:59:5...",{'doc-dateTime-max': '1961-03-03T23:59:59-05:0...,\n2. Memorandum from Schlesinger\n ...
2,volumes/frus1961-63v10-12mSupp.xml,frus1961-63v10-12mSupp,"<div frus:doc-dateTime-max=""1961-04-13T23:59:5...",{'doc-dateTime-max': '1961-04-13T23:59:59-05:0...,\n3. Memorandum from General Lemnitzer to McNa...
3,volumes/frus1961-63v10-12mSupp.xml,frus1961-63v10-12mSupp,"<div frus:doc-dateTime-max=""1961-07-18T23:59:5...",{'doc-dateTime-max': '1961-07-18T23:59:59-05:0...,\n4. Memorandum from Hilsman\n ...
4,volumes/frus1961-63v10-12mSupp.xml,frus1961-63v10-12mSupp,"<div frus:doc-dateTime-max=""1961-07-21T23:59:5...",{'doc-dateTime-max': '1961-07-21T23:59:59-05:0...,"\n5. Memorandum of conversation, July 21 betwe..."
...,...,...,...,...,...
309567,volumes/frus1872p2v4.xml,frus1872p2v4,"<div frus:doc-dateTime-max=""1872-12-01T23:55:5...",{'doc-dateTime-max': '1872-12-01T23:55:59-05:0...,\n\nExtracts from the continental press.\n\n\n...
309568,volumes/frus1872p2v4.xml,frus1872p2v4,"<div frus:doc-dateTime-max=""1872-06-06T23:59:5...",{'doc-dateTime-max': '1872-06-06T23:59:59-05:0...,\n[Untitled]\n\nExtract from the Journal\n ...
309569,volumes/frus1872p2v4.xml,frus1872p2v4,"<div frus:doc-dateTime-max=""1872-12-31T18:50:5...",{'doc-dateTime-max': '1872-12-31T18:50:59-05:0...,\nExtract from “La Question de l’Alabama et le...
309570,volumes/frus1872p2v4.xml,frus1872p2v4,"<div frus:doc-dateTime-max=""1872-12-31T18:42:5...",{'doc-dateTime-max': '1872-12-31T18:42:59-05:0...,\n“Quelques mots sur la phase nouvelle du diff...


In [10]:
df_more = pd.json_normalize(df.information)
df = pd.concat([df, df_more], axis=1)
df['id'] = df['source']+'/' + df['xml:id']

df = df.rename(columns={'doc-dateTime-max': 'date'})
df.date = df.date.apply(lambda x: x.split('T')[0])

df = df[['id','path','date','text','docs']]
df

,id,path,date,text,docs
0,frus1961-63v10-12mSupp/d1,volumes/frus1961-63v10-12mSupp.xml,1961-02-09,"\n1. Telegram 430 from La Paz, February 9Trans...","<div frus:doc-dateTime-max=""1961-02-09T23:59:5..."
1,frus1961-63v10-12mSupp/d2,volumes/frus1961-63v10-12mSupp.xml,1961-03-03,\n2. Memorandum from Schlesinger\n ...,"<div frus:doc-dateTime-max=""1961-03-03T23:59:5..."
2,frus1961-63v10-12mSupp/d3,volumes/frus1961-63v10-12mSupp.xml,1961-04-13,\n3. Memorandum from General Lemnitzer to McNa...,"<div frus:doc-dateTime-max=""1961-04-13T23:59:5..."
3,frus1961-63v10-12mSupp/d4,volumes/frus1961-63v10-12mSupp.xml,1961-07-18,\n4. Memorandum from Hilsman\n ...,"<div frus:doc-dateTime-max=""1961-07-18T23:59:5..."
4,frus1961-63v10-12mSupp/d5,volumes/frus1961-63v10-12mSupp.xml,1961-07-21,"\n5. Memorandum of conversation, July 21 betwe...","<div frus:doc-dateTime-max=""1961-07-21T23:59:5..."
...,...,...,...,...,...
309567,frus1872p2v4/d46,volumes/frus1872p2v4.xml,1872-12-01,\n\nExtracts from the continental press.\n\n\n...,"<div frus:doc-dateTime-max=""1872-12-01T23:55:5..."
309568,frus1872p2v4/d47,volumes/frus1872p2v4.xml,1872-06-06,\n[Untitled]\n\nExtract from the Journal\n ...,"<div frus:doc-dateTime-max=""1872-06-06T23:59:5..."
309569,frus1872p2v4/d48,volumes/frus1872p2v4.xml,1872-12-31,\nExtract from “La Question de l’Alabama et le...,"<div frus:doc-dateTime-max=""1872-12-31T18:50:5..."
309570,frus1872p2v4/d49,volumes/frus1872p2v4.xml,1872-12-31,\n“Quelques mots sur la phase nouvelle du diff...,"<div frus:doc-dateTime-max=""1872-12-31T18:42:5..."


In [14]:
df.to_csv('frus_text.csv', index=False)

# 3

In [58]:
df = pd.read_csv('frus_text.csv', header=0)
df

,id,path,date,text,docs
0,frus1961-63v10-12mSupp/d1,volumes/frus1961-63v10-12mSupp.xml,1961-02-09,"\n1. Telegram 430 from La Paz, February 9Trans...","<div frus:doc-dateTime-max=""1961-02-09T23:59:5..."
1,frus1961-63v10-12mSupp/d2,volumes/frus1961-63v10-12mSupp.xml,1961-03-03,\n2. Memorandum from Schlesinger\n ...,"<div frus:doc-dateTime-max=""1961-03-03T23:59:5..."
2,frus1961-63v10-12mSupp/d3,volumes/frus1961-63v10-12mSupp.xml,1961-04-13,\n3. Memorandum from General Lemnitzer to McNa...,"<div frus:doc-dateTime-max=""1961-04-13T23:59:5..."
3,frus1961-63v10-12mSupp/d4,volumes/frus1961-63v10-12mSupp.xml,1961-07-18,\n4. Memorandum from Hilsman\n ...,"<div frus:doc-dateTime-max=""1961-07-18T23:59:5..."
4,frus1961-63v10-12mSupp/d5,volumes/frus1961-63v10-12mSupp.xml,1961-07-21,"\n5. Memorandum of conversation, July 21 betwe...","<div frus:doc-dateTime-max=""1961-07-21T23:59:5..."
...,...,...,...,...,...
309567,frus1872p2v4/d46,volumes/frus1872p2v4.xml,1872-12-01,\n\nExtracts from the continental press.\n\n\n...,"<div frus:doc-dateTime-max=""1872-12-01T23:55:5..."
309568,frus1872p2v4/d47,volumes/frus1872p2v4.xml,1872-06-06,\n[Untitled]\n\nExtract from the Journal\n ...,"<div frus:doc-dateTime-max=""1872-06-06T23:59:5..."
309569,frus1872p2v4/d48,volumes/frus1872p2v4.xml,1872-12-31,\nExtract from “La Question de l’Alabama et le...,"<div frus:doc-dateTime-max=""1872-12-31T18:50:5..."
309570,frus1872p2v4/d49,volumes/frus1872p2v4.xml,1872-12-31,\n“Quelques mots sur la phase nouvelle du diff...,"<div frus:doc-dateTime-max=""1872-12-31T18:42:5..."


In [59]:
def clean_text(x):
    x = re.sub('\n  +',' ',x)
    x = re.sub(' +',' ',x)
    x = re.sub("^\s+|\s+$", "", x, flags=re.UNICODE)
    return x

df['text'] = df['text'].parallel_apply(clean_text).values

In [66]:
df = df.sort_values('date').reset_index(drop=True)
df

,id,path,date,text,docs
0,frus1872p2v5/d4,volumes/frus1872p2v5.xml,1620-11-03,No. 3.\n\nExtract from the patent granted by J...,"<div frus:doc-dateTime-max=""1620-11-03T23:59:5..."
1,frus1872p2v5/d5,volumes/frus1872p2v5.xml,1628-03-04,[Untitled]\n\nExtract from the charter of Mass...,"<div frus:doc-dateTime-max=""1628-03-04T23:59:5..."
2,frus1872p2v5/d6,volumes/frus1872p2v5.xml,1663-03-24,[Untitled]\n[undated]\n*Extract from the old p...,"<div frus:doc-dateTime-max=""1663-03-24T23:55:5..."
3,frus1902app2/d26,volumes/frus1902app2.xml,1757-12-31,extracts from the work entitled “account of ca...,"<div frus:doc-dateTime-max=""1757-12-31T23:59:5..."
4,frus1902app2/d7,volumes/frus1902app2.xml,1759-12-31,Exhibit B.\n\n[1741–1759]\n\nRésumé of litigat...,"<div frus:doc-dateTime-max=""1759-12-31T23:59:5..."
...,...,...,...,...,...
309567,frus1981-88v06/d54,volumes/frus1981-88v06.xml,1989-01-08,"54. Editorial Note\nOn June 3, 1987, President...","<div frus:doc-dateTime-max=""1989-01-08T16:05:0..."
309568,frus1981-88v06/d9,volumes/frus1981-88v06.xml,1989-01-08,"9. Editorial Note\nOn November 25, 1986, the P...","<div frus:doc-dateTime-max=""1989-01-08T16:05:0..."
309569,frus1981-88v06/d2,volumes/frus1981-88v06.xml,1989-01-08,2. Editorial Note\nIn his diary entry for Octo...,"<div frus:doc-dateTime-max=""1989-01-08T16:05:0..."
309570,frus1981-88v06/d48,volumes/frus1981-88v06.xml,1989-01-08,"48. Editorial Note\nOn April 16, 1987, Secreta...","<div frus:doc-dateTime-max=""1989-01-08T16:05:0..."


In [ ]:
df.text.str.len().describe()

In [14]:
pandarallel.initialize(nb_workers=38) 
df.text = df.text.parallel_apply(lambda x: unicodetoascii(x))
df.text = df.text.parallel_apply(lambda x: unicodedata.normalize("NFKD", x))
df = df[df.text.str.len()>200]

df[['id','text','date']].to_csv('frus_1620_1989.csv', index=False)

INFO: Pandarallel will run on 38 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


# 4  history filtering, after world war II.

In [8]:
df= pd.read_csv('frus_1620_1989.csv', header=0)
df

,id,text,date
0,frus1872p2v5/d4,No. 3.\n\nExtract from the patent granted by J...,1620-11-03
1,frus1872p2v5/d5,[Untitled]\n\nExtract from the charter of Mass...,1628-03-04
2,frus1872p2v5/d6,[Untitled]\n[undated]\n*Extract from the old p...,1663-03-24
3,frus1902app2/d26,extracts from the work entitled “account of ca...,1757-12-31
4,frus1902app2/d7,Exhibit B.\n\n[1741–1759]\n\nRésumé of litig...,1759-12-31
...,...,...,...
308605,frus1981-88v06/d22,"22. Editorial Note\nOn February 28, 1987, Sovi...",1989-01-08
308606,frus1981-88v06/d14,"14. Editorial Note\nOn January 21, 1987, Presi...",1989-01-08
308607,frus1981-88v06/d155,"155. Editorial Note\nOn May 25, 1988, Presiden...",1989-01-08
308608,frus1981-88v06/d54,"54. Editorial Note\nOn June 3, 1987, President...",1989-01-08


In [16]:
df = df[df.date>'1945-09-02']
df

,id,text,date
173786,frus1945v07/d409,893.00/9–445: Telegram\n\nThe Ambassador in Ch...,1945-09-03
173787,frus1945v05/d449,871.00/9–345: Telegram\n\nThe Secretary of Sta...,1945-09-03
173788,frus1945v07/d313,893.00/9–345: Telegram\n\nThe Ambassador in th...,1945-09-03
173789,frus1945v04/d321,874.00/9–345: Telegram\n\nThe United States Re...,1945-09-03
173790,frus1945v08/d204,868.00/9–345: Telegram\n\nThe Ambassador in Gr...,1945-09-03
...,...,...,...
308605,frus1981-88v06/d22,"22. Editorial Note\nOn February 28, 1987, Sovi...",1989-01-08
308606,frus1981-88v06/d14,"14. Editorial Note\nOn January 21, 1987, Presi...",1989-01-08
308607,frus1981-88v06/d155,"155. Editorial Note\nOn May 25, 1988, Presiden...",1989-01-08
308608,frus1981-88v06/d54,"54. Editorial Note\nOn June 3, 1987, President...",1989-01-08


In [17]:
df.to_csv('frus_1945_1989.csv', index=False)

In [20]:
pandarallel.initialize(nb_workers=38) 
num_chunks = 50
df_save = np.array_split(df, num_chunks)
save_chunks = lambda x: df_save[x].to_csv('frus/%03d.csv'%x, index=False)     
_ = pd.Series(range(num_chunks)).parallel_apply(save_chunks) 

INFO: Pandarallel will run on 38 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [24]:
get_size = lambda x: get_csv_size('frus/%03d.csv'%x)
values = pd.Series(range(num_chunks)).parallel_apply(get_size) 
values

0      8.6
1      9.5
2     11.0
3     11.8
4     10.3
5      9.5
6     10.1
7      9.7
8      9.7
9      9.0
10     9.2
11     9.0
12    12.0
13    12.5
14    13.4
15    13.2
16    11.9
17    14.4
18    14.5
19    14.3
20    13.3
21    11.1
22    10.3
23    14.1
24    14.5
25    14.6
26    12.6
27    14.2
28    15.7
29    12.2
30    16.4
31    15.1
32    14.8
33    12.5
34    13.5
35    13.9
36    14.3
37    14.0
38    13.4
39    17.6
40    18.2
41    19.3
42    22.4
43    20.6
44    21.2
45    20.5
46    20.9
47    19.2
48    17.5
49    20.7
dtype: float64

In [25]:
values.sum()

702.2

# 5  split to 40 chunks

In [8]:
num_chunks = 40 

In [9]:
df1=pd.read_csv('frus_1945_1989.csv',header=[0])

# df_all = np.array_split(df1, num_chunks)

df_save = np.array_split(df1, num_chunks)
save_chunks = lambda x: df_save[x].to_csv('result/csv/frus_%03d.csv'%x, index=False)

pandarallel.initialize(progress_bar=True)       
_ = pd.Series(range(num_chunks)).parallel_apply(save_chunks) 

INFO: Pandarallel will run on 36 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


# 6 save to json and tar

In [10]:
num_chunks = 40

df3 = pd.DataFrame(columns=['filename'])
df3['filename'] = ['frus_%03d.csv'%idx for idx in range(num_chunks)]

df3['path'] = 'result/csv/' + df3.filename
df3['json_file'] = df3['filename'].apply(lambda x: x.replace('.csv', '.json'))
df3['tar_file'] = 'result/tar/' + df3.json_file+'.tar.gz'
df3.json_file = 'result/json/'+df3.json_file
df3

,filename,path,json_file,tar_file
0,frus_000.csv,result/csv/frus_000.csv,result/json/frus_000.json,result/tar/frus_000.json.tar.gz
1,frus_001.csv,result/csv/frus_001.csv,result/json/frus_001.json,result/tar/frus_001.json.tar.gz
2,frus_002.csv,result/csv/frus_002.csv,result/json/frus_002.json,result/tar/frus_002.json.tar.gz
3,frus_003.csv,result/csv/frus_003.csv,result/json/frus_003.json,result/tar/frus_003.json.tar.gz
4,frus_004.csv,result/csv/frus_004.csv,result/json/frus_004.json,result/tar/frus_004.json.tar.gz
5,frus_005.csv,result/csv/frus_005.csv,result/json/frus_005.json,result/tar/frus_005.json.tar.gz
6,frus_006.csv,result/csv/frus_006.csv,result/json/frus_006.json,result/tar/frus_006.json.tar.gz
7,frus_007.csv,result/csv/frus_007.csv,result/json/frus_007.json,result/tar/frus_007.json.tar.gz
8,frus_008.csv,result/csv/frus_008.csv,result/json/frus_008.json,result/tar/frus_008.json.tar.gz
9,frus_009.csv,result/csv/frus_009.csv,result/json/frus_009.json,result/tar/frus_009.json.tar.gz


In [17]:
def convert_json_tar(filename, json_file, tar_file):
    df1 = pd.read_csv(filename, header=[0])
    
    df1.to_json(json_file, orient="records", lines=True)
    
    with tarfile.open(tar_file, "w:gz") as tar:
        tar.add(json_file, arcname=os.path.basename(json_file))

        
pandarallel.initialize(nb_workers=32, progress_bar=True) 
_ = df3.parallel_apply(lambda x: convert_json_tar(x['path'], x['json_file'], x['tar_file']), axis=1)

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
